In [1]:
import numpy as np
import pandas as pd

import hdbscan

import seaborn as sns
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show, output_notebook
import matplotlib as mpl
from bokeh.layouts import row,column
from bokeh.models import HoverTool
from bokeh.models.sources import ColumnDataSource
%matplotlib inline

In [39]:
peakx_ind = np.load('peakx_ind.npy')  #indexes of l
peaky_ind = np.load('peaky_ind.npy')  #indexes of m 
integration_ind = np.load('timearray_ind.npy')
dm_ind = np.load('dmarray_ind.npy')
dt_ind = np.load('dtind.npy')
snr = np.load('snr.npy')

In [40]:
time_ind = np.multiply(integration_ind,np.power(2,dt_ind))

In [41]:
d = {'l': peakx_ind, 'm': peaky_ind, 'dm': dm_ind, 'time': time_ind, 'snr': snr}
df = pd.DataFrame(data=d)

In [42]:
max(df.snr)

22.809436798095703

In [43]:
df.shape

(222, 5)

In [44]:
# Clustering! 
clusterer = hdbscan.HDBSCAN(min_cluster_size=10, min_samples = 5, cluster_selection_method = 'eom', allow_single_cluster = True).fit(df)
np.max(clusterer.labels_+1) #number of clusters

4

In [45]:
color_palette = sns.color_palette('deep', np.max(clusterer.labels_) + 1) #get a color palette with number of colors = number of clusters
cluster_colors = [color_palette[x] if x >= 0
                  else (0.5, 0.5, 0.5)
                  for x in clusterer.labels_]    #assigning each cluster a color, and making a list

cluster_member_colors = [sns.desaturate(x, p) for x, p in
                         zip(cluster_colors, clusterer.probabilities_)]

#plt.scatter(df.x,df.y, s=50, linewidth=0, c=cluster_member_colors, alpha=0.4)
#plt.xlim(df.x.min(axis=0),df.x.max(axis=0))
#plt.ylim(df.y.min(axis=0), df.y.max(axis=0))

In [47]:
cluster_colors = list(map(mpl.colors.rgb2hex, cluster_member_colors)) #converting sns colors to hex for bokeh

# Bokeh Plotting

In [48]:
width = 450
height = 350
output_notebook()

TOOLS = 'crosshair, box_zoom, reset, box_select, tap, hover, wheel_zoom'
data = dict(l= peakx_ind, m= peaky_ind, dm= dm_ind, time= time_ind, snr= snr, colors = cluster_colors)
source=ColumnDataSource(data=data)


p = figure(title="m vs l", x_axis_label='l', y_axis_label='m',plot_width=width, plot_height=height, tools = TOOLS)
p.circle(x='l',y='m', size='snr', line_width = 1, color = 'colors', fill_alpha=0.3, source = source) # linewidth=0,
#p.circle(x=df.l,y=df.m, size=5, line_width = 1, color = cluster_colors, fill_alpha=0.5) # linewidth=0,
hover = p.select(dict(type=HoverTool))
hover.tooltips = [("m", "@m"), ("l", "@l"), ("time", "@time"), ("DM", "@dm"), ("SNR", "@snr")]

#p.circle(x,y, size=5, line_width = 1, color = colors)#, , fill_alpha=1) # linewidth=0,
#p.circle(x="x", y="y", source=source, size=7, color="color", line_color=None, fill_alpha="alpha")
p2 = figure(title="DM vs time", x_axis_label='time', y_axis_label='DM',plot_width=width, plot_height=height, tools = TOOLS)
p2.circle(x='time',y='dm', size='snr', line_width = 1, color = 'colors', fill_alpha=0.3, source=source) # linewidth=0,
hover = p2.select(dict(type=HoverTool))
hover.tooltips = [("m", "@m"), ("l", "@l"), ("time", "@time"), ("DM", "@dm"), ("SNR", "@snr")]


p3 = figure(title="DM vs l", x_axis_label='l', y_axis_label='DM',plot_width=width, plot_height=height, tools = TOOLS)
p3.circle(x='l',y='dm', size='snr', line_width = 1, color = 'colors', fill_alpha=0.3, source=source) # linewidth=0,
hover = p3.select(dict(type=HoverTool))
hover.tooltips = [("m", "@m"), ("l", "@l"), ("time", "@time"), ("DM", "@dm"), ("SNR", "@snr")]


p4 = figure(title="time vs l", x_axis_label='l', y_axis_label='time',plot_width=width, plot_height=height, tools = TOOLS)
p4.circle(x='l',y='time', size='snr', line_width = 1, color = 'colors', fill_alpha=0.3, source=source) # linewidth=0,
hover = p4.select(dict(type=HoverTool))
hover.tooltips = [("m", "@m"), ("l", "@l"), ("time", "@time"), ("DM", "@dm"), ("SNR", "@snr")]


# show the results
show(column(row(p,p2),row(p3,p4)))

Loading BokehJS ...